In [3]:
def get_winner(you, boss): # (health, damage, armor)
    # you = (8,5,5), boss = (12,7, 2)
    your_damage = max(1,you[1] - boss[2])
    boss_damage = max(1,boss[1] - you[2])
    
    while True:         
        boss = (boss[0] - your_damage, boss[1], boss[2])
        you = (you[0] - boss_damage, you[1], you[2])

        if boss[0] <= 0:

            return 'you'
        elif you[0] <= 0: 
            return 'boss'

def get_outcomes():         
    results = list()        
    for weapon in new_weapons: 
        for armor in new_armor: 
            for ring1 in new_rings: 
                for ring2 in new_rings: 
                    my_damage = weapon[2] + armor[2] + ring1[2] + ring2[2]
                    my_armor = weapon[3] + armor[3] + ring1[3] + ring2[3]
                    my_cost = weapon[1] + armor[1] + ring1[1] + ring2[1]

                    me = (100, my_damage, my_armor)        
                    boss = (104, 8, 1)   # health, damage, armor

                    results.append((get_winner(me, boss), my_cost))
    return results
results = get_outcomes()                    
print('part 1:', min([x[1] for x in results if x[0] == 'you']))
print('part 2:', max([x[1] for x in results if x[0] == 'boss']))

part 1: 78
part 2: 148


In [31]:
from collections import namedtuple
from types import SimpleNamespace
import copy
# Effect = namedtuple("Effect", "name turnsleft")
# Player = namedtuple("Player", "hitpoint damage armor mana effects")


In [52]:
mana_cost = {
    "magicmissile": 53, 
    "drain": 73, 
    "shield": 113, 
    "poison": 173, 
    "recharge": 229
}
you = SimpleNamespace(hitpoint=50, damage=0, armor=0, mana=500, effects=[])
boss = SimpleNamespace(hitpoint=71, damage=10, armor=0, mana=0, effects=[])

expand_stack = [(you, boss, 0, True)]
# State is the player and the boss battling it out, the total amount of mana spent, and whether it is your turn

hard_mode = True

lowest_spent = 9999999999
while expand_stack: 
#     print(len(expand_stack), expand_stack[0])
    player, boss, total_spent, your_turn = expand_stack.pop(0)
    
    boss_attack = boss.damage
    player_defense = player.armor

    for effect in player.effects: 
        if effect.name == "shield":
            player_defense += 7
        if effect.name == "recharge":
            player.mana += 101

    for effect in boss.effects: 
        if effect.name == "poison":
            boss.hitpoint -= 3

    if hard_mode and your_turn: 
        player.hitpoint -= 1
        
    if boss.hitpoint <= 0: 
        lowest_spent = min(lowest_spent, total_spent)
        print("Player wins", total_spent, "lowest spent:", lowest_spent)
        continue
    elif player.hitpoint <= 0: 
        continue
        
    # Reduce the turns left per player
    for effect in player.effects: 
        effect.turnsleft -= 1
    for effect in boss.effects: 
        effect.turnsleft -= 1

    boss.effects = [effect for effect in boss.effects if effect.turnsleft > 0]
    player.effects = [effect for effect in player.effects if effect.turnsleft > 0]
    
    
    if your_turn: 
        for spell in ["magicmissile", "drain", "shield", "poison", "recharge"]:
            if player.mana < mana_cost[spell]:
                continue
                
                    
            # Here we expand our option tree! 
            new_player, new_boss = copy.deepcopy(player), copy.deepcopy(boss)
            new_player.mana -= mana_cost[spell]
            
            if spell == 'magicmissile':
                new_boss.hitpoint -= 4
            elif spell == 'drain': 
                new_boss.hitpoint -= 2
                new_player.hitpoint += 2
            elif spell == 'shield': 
                new_player.effects.append(SimpleNamespace(name=spell, turnsleft=6))
            elif spell == 'poison': 
                new_boss.effects.append(SimpleNamespace(name=spell, turnsleft=6))
            elif spell == 'recharge': 
                new_player.effects.append(SimpleNamespace(name=spell, turnsleft=5))
            else:
                print('error"')
                
            player_effect_names = [e.name for e in new_player.effects]
            boss_effect_names = [e.name for e in new_boss.effects]
            if len(set(player_effect_names)) == len(player_effect_names) and len(set(boss_effect_names)) == len(boss_effect_names):
                expand_stack.append((new_player, new_boss, total_spent + mana_cost[spell], False))
    else: 
        damage = max(1, boss_attack - player_defense)
        player.hitpoint -= damage
        expand_stack.append((player, boss, total_spent, True))


Player wins 1937 lowest spent: 1937
Player wins 1957 lowest spent: 1937
Player wins 1957 lowest spent: 1937
